In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
import torch
from sklearn.preprocessing import MinMaxScaler
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D, Dataset1D
import threading
import json

/Users/mkhoatd/Developer/gdm-network/ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#%%




diffusion = None
trainer = None
scaler = None
encoder = None
data_cols = None
res_cached = []

infer_mutex = threading.Lock()


def load_model(input, weights):
    global trainer
    global diffusion
    global scaler
    global encoder
    global data_cols
    data = pd.read_csv(input)
    data = data.drop(columns = 'id')
    #Create object for one-hot encoding
    encoder=ce.OneHotEncoder(cols=['state','proto','attack_cat','service'],handle_unknown='return_nan',return_df=True,use_cat_names=True)
    #Fit and transform Data
    data_encoded = encoder.fit_transform(data)
    scaler = MinMaxScaler()
    dataset1_norm = scaler.fit_transform(data_encoded.values)
    dataset1 = torch.Tensor(dataset1_norm)
    dataset1 = torch.unsqueeze(dataset1,1)
    shape0 = dataset1.shape[0]
    shape1 = dataset1.shape[1]
    dataset1 = torch.concat([dataset1,torch.zeros(shape0,shape1,3)],dim=2)
    data_cols = data_encoded.columns
    ######################################################################################


    model = Unet1D(
        dim = 16,
        dim_mults = (1, 2, 4, 8),
        channels = 1
        )

    diffusion = GaussianDiffusion1D(
        model,
        seq_length = 208,
        timesteps = 1000,
        objective = 'pred_v'
        )

    dataset = Dataset1D(dataset1)  
    amp = False if not torch.cuda.is_available() else True

    trainer = Trainer1D(
        diffusion,
        dataset = dataset,
        train_batch_size = 1,
        train_lr = 8e-5,
        train_num_steps = 10000,         # total training steps
        gradient_accumulate_every = 2,    # gradient accumulation steps
        ema_decay = 0.995,                # exponential moving average decay
        amp = amp,                       # turn on mixed precision
        )
    trainer.load(weights)


def infer(size=100):
    # Acquire mutex before running inference
    with infer_mutex:
        if diffusion is None:
            input = 'UNSW_NB15_testing-set.csv'
            weights = 'network-traffic'
            # print(GDM_model(input,weights))
            load_model(input, weights)
        sampled_seq = diffusion.sample(batch_size = size)
        sampled_seq_sq = torch.squeeze(sampled_seq, 1)[:,0:-3]
        sampled_seq_sq = scaler.inverse_transform(sampled_seq_sq.cpu())
        a = sampled_seq_sq
        #### 
        argmax = np.argmax(a[:,1:134],axis=1) + 1
        for i in range(0,a.shape[0]):
            for j in range(0,a.shape[1]):
                if j == argmax[i] and j >= 1 and j <= 133:
                    a[i][j] = 1
                elif j != argmax[i] and j >= 1 and j <= 133:
                    a[i][j] = 0
        ###
        argmax = np.argmax(a[:,134:147],axis=1) + 134
        for i in range(0,a.shape[0]):
            for j in range(0,a.shape[1]):
                if j == argmax[i] and j >= 134 and j <= 146:
                    a[i][j] = 1
                elif j != argmax[i] and j >= 134 and j <= 146:
                    a[i][j] = 0
        ###
        argmax = np.argmax(a[:,147:156],axis=1) + 147
        for i in range(0,a.shape[0]):
            for j in range(0,a.shape[1]):
                if j == argmax[i] and j >= 147 and j <= 155:
                    a[i][j] = 1
                elif j != argmax[i] and j >= 147 and j <= 155:
                    a[i][j] = 0
        ###
        argmax = np.argmax(a[:,194:204],axis=1) + 194
        for i in range(0,a.shape[0]):
            for j in range(0,a.shape[1]):
                if j == argmax[i] and j >= 194 and j <= 203:
                    a[i][j] = 1
                elif j != argmax[i] and j >= 194 and j <= 203:
                    a[i][j] = 0
        sampled_seq_df = pd.DataFrame(a)
        sampled_seq_df.columns = data_cols
        data_decoded = encoder.inverse_transform(sampled_seq_df)
        columns_to_convert = ['spkts', 'dpkts', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'swin', 'stcpb', 'dtcpb', 'dwin', 'smean', 'dmean', 'trans_depth',
                         'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login',
                          'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label']

        data_decoded[columns_to_convert] = data_decoded[columns_to_convert].astype(float).round().astype(int)

        return data_decoded


def GDM_model(input,weights):

    ############################### Pre-processing ######################################
    data = pd.read_csv(input)
    data = data.drop(columns = 'id')
    #Create object for one-hot encoding
    encoder=ce.OneHotEncoder(cols=['state','proto','attack_cat','service'],handle_unknown='return_nan',return_df=True,use_cat_names=True)
    #Fit and transform Data
    data_encoded = encoder.fit_transform(data)
    scaler = MinMaxScaler()
    dataset1_norm = scaler.fit_transform(data_encoded.values)
    dataset1 = torch.Tensor(dataset1_norm)
    dataset1 = torch.unsqueeze(dataset1,1)
    shape0 = dataset1.shape[0]
    shape1 = dataset1.shape[1]
    dataset1 = torch.concat([dataset1,torch.zeros(shape0,shape1,3)],dim=2)
    ######################################################################################


    model = Unet1D(
        dim = 16,
        dim_mults = (1, 2, 4, 8),
        channels = 1
        )

    diffusion = GaussianDiffusion1D(
        model,
        seq_length = 208,
        timesteps = 1000,
        objective = 'pred_v'
        )

    dataset = Dataset1D(dataset1)  

    trainer = Trainer1D(
        diffusion,
        dataset = dataset,
        train_batch_size = 1,
        train_lr = 8e-5,
        train_num_steps = 10000,         # total training steps
        gradient_accumulate_every = 2,    # gradient accumulation steps
        ema_decay = 0.995,                # exponential moving average decay
        amp = False,                       # turn on mixed precision
        )
    trainer.load(weights)

    ####################################### Post processing ##############################################
    sampled_seq = diffusion.sample(batch_size = 100)
    sampled_seq_sq = torch.squeeze(sampled_seq, 1)[:,0:-3]
    sampled_seq_sq = scaler.inverse_transform(sampled_seq_sq.cpu())
    a = sampled_seq_sq
    #### 
    argmax = np.argmax(a[:,1:134],axis=1) + 1
    for i in range(0,a.shape[0]):
        for j in range(0,a.shape[1]):
            if j == argmax[i] and j >= 1 and j <= 133:
                a[i][j] = 1
            elif j != argmax[i] and j >= 1 and j <= 133:
                a[i][j] = 0
    ###
    argmax = np.argmax(a[:,134:147],axis=1) + 134
    for i in range(0,a.shape[0]):
        for j in range(0,a.shape[1]):
            if j == argmax[i] and j >= 134 and j <= 146:
                a[i][j] = 1
            elif j != argmax[i] and j >= 134 and j <= 146:
                a[i][j] = 0
    ###
    argmax = np.argmax(a[:,147:156],axis=1) + 147
    for i in range(0,a.shape[0]):
        for j in range(0,a.shape[1]):
            if j == argmax[i] and j >= 147 and j <= 155:
                a[i][j] = 1
            elif j != argmax[i] and j >= 147 and j <= 155:
                a[i][j] = 0
    ###
    argmax = np.argmax(a[:,194:204],axis=1) + 194
    for i in range(0,a.shape[0]):
        for j in range(0,a.shape[1]):
            if j == argmax[i] and j >= 194 and j <= 203:
                a[i][j] = 1
            elif j != argmax[i] and j >= 194 and j <= 203:
                a[i][j] = 0
    sampled_seq_df = pd.DataFrame(a)
    sampled_seq_df.columns = data_encoded.columns
    data_decoded = encoder.inverse_transform(sampled_seq_df)
    columns_to_convert = ['spkts', 'dpkts', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'swin', 'stcpb', 'dtcpb', 'dwin', 'smean', 'dmean', 'trans_depth',
                     'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login',
                      'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label']

    data_decoded[columns_to_convert] = data_decoded[columns_to_convert].astype(float).round().astype(int)

    return data_decoded


In [3]:
input = 'UNSW_NB15_testing-set.csv'
weights = 'network-traffic'
# print(GDM_model(input,weights))
load_model(input, weights)

loading from version 2.1.1


In [8]:
res = infer(2)

sampling loop time step: 100%|██████████| 1000/1000 [00:39<00:00, 25.30it/s]


In [10]:
res.columns

Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')

In [11]:
with open('res.json', 'w') as f:
    json.dump(res.to_dict(orient='records'), f)


In [8]:
d = res.to_dict()

In [9]:
d.keys()

dict_keys(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'])